In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor 

from sklearn.linear_model import LinearRegression,Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Инициализируем все известные генераторы случаынйх чисел / Setting all known random seeds
my_code = "Бевз А."
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

In [3]:
# Читаем данные из файла
train_data = pd.read_csv("datasets/rus_stocks_funds_train.csv")

In [4]:
train_data.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,7.210000e+09,2.870000e+09,110809.6875,3190.000000,9.870000,0.27
1,1.060000e+10,1.130000e+09,318000.0000,28230.000000,13.910000,0.30
2,2.249000e+10,1.027000e+10,329570.0000,61044.948718,4.080000,0.25
3,4.995200e+08,1.964200e+08,59380.0000,81.000000,2.910000,1.31
4,9.520000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.35


In [5]:
# Определим размер валидационной выборки
val_size = round(0.2*len(train_data))
print(val_size)

66


In [6]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, test_size=val_size, random_state=random_state)
print(len(train), len(val))

263 66


In [7]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
y_column = 'Debt/Assets'
x_columns = [e for e in train_data.columns if e != y_column]

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), x_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Total issued ordinary shares',
                                  'Number of shares available for sale',
                                  'Number of employees',
                                  'Number of shareholders', 'Price/Profit'])])

In [8]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

In [9]:
# Устанавливаем названия столбцов
column_names = list(x_columns) + [y_column]
sc_train.columns = column_names
sc_val.columns = column_names

In [10]:
sc_train

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,0.032599,0.025894,0.233354,0.149645,0.076315,0.21
1,0.000817,0.000223,0.676068,0.069197,0.056029,0.30
2,0.000011,0.000012,0.233354,0.094595,0.042900,0.55
3,0.000057,0.000082,0.233354,0.096115,0.012558,0.16
4,0.000461,0.000171,0.117735,0.020068,0.022745,0.30
...,...,...,...,...,...,...
258,0.000009,0.025894,0.233354,0.149645,0.041802,0.47
259,0.000053,0.000025,0.233354,0.001689,0.009836,0.03
260,0.003351,0.007035,0.233354,0.076527,0.023843,0.00
261,0.000013,0.000015,0.233354,0.000015,0.007026,0.00


In [11]:
# Отберем необходимые параметры
x_train = sc_train[x_columns]
x_val = sc_val[x_columns]

y_train = (sc_train[y_column].values).flatten()
y_val = (sc_val[y_column].values).flatten()

In [12]:
# Создадим простую модель логистической регрессии
r_models = []

r_models.append(ElasticNet(alpha=0.5))
r_models.append(ElasticNet(alpha=0.5, l1_ratio=0.75))
r_models.append(LinearRegression())
r_models.append(Lasso())

r_models.append(DecisionTreeRegressor())

r_models.append(SVR())

r_models.append(Ridge())

r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

r_models.append(SVR(kernel='linear'))
r_models.append(SVR(kernel='poly'))
r_models.append(SVR(kernel='rbf'))
r_models.append(SVR(kernel='sigmoid'))

r_models.append(DecisionTreeRegressor(criterion='squared_error'))
r_models.append(DecisionTreeRegressor(criterion='friedman_mse'))
r_models.append(DecisionTreeRegressor(criterion='absolute_error'))

In [13]:
r_models

[ElasticNet(alpha=0.5),
 ElasticNet(alpha=0.5, l1_ratio=0.75),
 LinearRegression(),
 Lasso(),
 DecisionTreeRegressor(),
 SVR(),
 Ridge(),
 KNeighborsRegressor(),
 KNeighborsRegressor(n_neighbors=10),
 KNeighborsRegressor(n_neighbors=15),
 SVR(kernel='linear'),
 SVR(kernel='poly'),
 SVR(),
 SVR(kernel='sigmoid'),
 DecisionTreeRegressor(),
 DecisionTreeRegressor(criterion='friedman_mse'),
 DecisionTreeRegressor(criterion='absolute_error')]

In [14]:
# Обучим модель
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

ElasticNet(alpha=0.5)
ElasticNet(alpha=0.5, l1_ratio=0.75)
LinearRegression()
Lasso()
DecisionTreeRegressor()
SVR()
Ridge()
KNeighborsRegressor()
KNeighborsRegressor(n_neighbors=10)
KNeighborsRegressor(n_neighbors=15)
SVR(kernel='linear')
SVR(kernel='poly')
SVR()
SVR(kernel='sigmoid')
DecisionTreeRegressor()
DecisionTreeRegressor(criterion='friedman_mse')
DecisionTreeRegressor(criterion='absolute_error')


In [15]:
# Проверим работу обученной модели на валидационной выборке
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

ElasticNet(alpha=0.5) 	 0.061358312098423864
ElasticNet(alpha=0.5, l1_ratio=0.75) 	 0.061358312098423864
LinearRegression() 	 0.07016785123729768
Lasso() 	 0.061358312098423864
DecisionTreeRegressor() 	 0.04261969696969697
SVR() 	 0.06347830949970229
Ridge() 	 0.06779048491861722
KNeighborsRegressor() 	 0.06746141770597214
KNeighborsRegressor(n_neighbors=10) 	 0.06688579838008012
KNeighborsRegressor(n_neighbors=15) 	 0.06822344286519483
SVR(kernel='linear') 	 0.06392821694804396
SVR(kernel='poly') 	 1.6824460395958345
SVR() 	 0.06347830949970229
SVR(kernel='sigmoid') 	 52.38524087395063
DecisionTreeRegressor() 	 0.036843939393939394
DecisionTreeRegressor(criterion='friedman_mse') 	 0.037413636363636364
DecisionTreeRegressor(criterion='absolute_error') 	 0.048583484086747175


In [16]:
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

DecisionTreeRegressor()

In [17]:
val_pred

array([0.1       , 0.1       , 0.24      , 0.41      , 1.06      ,
       0.        , 0.22      , 0.61      , 0.25      , 0.38      ,
       0.14      , 0.27      , 0.51      , 0.83      , 0.68      ,
       0.31      , 0.3       , 0.24      , 0.32      , 0.61      ,
       0.24      , 0.2       , 0.45      , 0.22      , 0.06      ,
       0.21      , 0.33      , 0.09      , 1.06      , 1.06      ,
       0.28      , 0.25      , 0.33224939, 0.21      , 0.08      ,
       0.15      , 0.62      , 1.52      , 0.        , 0.48      ,
       0.17      , 0.        , 0.16      , 0.19      , 0.        ,
       0.12      , 0.19      , 0.21      , 0.19      , 0.22      ,
       0.19      , 0.18      , 0.21      , 0.18      , 0.14      ,
       0.31      , 0.17      , 0.17      , 0.52      , 0.09      ,
       0.21      , 0.2       , 0.71      , 0.2       , 0.7       ,
       0.06      ])

In [18]:
test = pd.read_csv("datasets/rus_stocks_funds_test.csv")

In [19]:
test[y_column] = 0.0

In [20]:
test

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.0
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.0
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.0
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.0
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.0
...,...,...,...,...,...,...
77,1.528600e+08,1.309960e+11,110809.6875,61044.948718,6.270000,0.0
78,1.295000e+08,2.634000e+07,110809.6875,25.000000,15.525191,0.0
79,2.246000e+10,0.000000e+00,281550.0000,61044.948718,6.340000,0.0
80,1.295000e+08,3.371000e+07,110809.6875,25.000000,6.320000,0.0


In [21]:
sc_test = pd.DataFrame(ct.transform(test))
sc_test.columns = column_names

In [22]:
x_test = sc_test[x_columns]

In [23]:
test[y_column] = model.predict(x_test)

In [24]:
test.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.200
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.120
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.270
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.480
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.705


In [25]:
test.to_csv('task5_results/Бевз А..csv', index=False)